|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating token embeddings<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge: Coloring cosine similarity<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [ ]:
from transformers import BertTokenizer, BertModel

# load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

embeddings = model.embeddings.word_embeddings.weight.detach().numpy()

In [ ]:
# code to calculate the width of a letter
fig,ax = plt.subplots(figsize=(10,2))

# draw a text object
temp_text = ax.text(0,0,'n',fontsize=12,fontfamily='monospace')

# Get its bounding box in display coordinates
bbox = temp_text.get_window_extent(renderer=fig.canvas.get_renderer())

# convert from display to axis coordinates
inv = ax.transAxes.inverted()
bbox_axes = inv.transform([[bbox.x0,bbox.y0], [bbox.x1,bbox.y1]])
en_width = bbox_axes[1,0] - bbox_axes[0,0] # bbox is [(x0,y0),(x1,y1)]

plt.close(fig)
en_width

# **IMPORTANT NOTE**:
## The following code cell is for video "CodeChallenge: Can random embeddings be interpreted?"

## Leave it commented until that video!


In [ ]:
# ### --- random shuffling, option 1

# # in-place shuffling, but only the rows
# np.random.shuffle(embeddings)

# # transpose and shuffle the columns, then transpose back
# embeddings = embeddings.T
# np.random.shuffle(embeddings)
# embeddings = embeddings.T

# ### --- random shuffling, option 2

# # get permuted indices
# randindices = np.random.permutation(np.prod(embeddings.shape))

# # randomize the vectorized matrix
# embeddings_flat = embeddings.flatten()[randindices]

# # reshape back to 2D
# embeddings = embeddings_flat.reshape(embeddings.shape)

# Exercise 1: Embeddings vector magnitudes

In [ ]:
# tokenize text
# https://en.wikipedia.org/wiki/Tbilisi
text = "Because of its location at the crossroads between Europe and Asia, and its proximity to the lucrative Silk Road, throughout history, Tbilisi has been a point of contention among various global powers. To this day, the city's location ensures its position as an important transit route for energy and trade projects. Tbilisi's history is reflected in its architecture, which is a mix of medieval, neoclassical, Beaux Arts, Art Nouveau, Stalinist, and Modern structures."
tokens = tokenizer.encode(text)[1:-1]

# get all magnitudes
magnitudes = np.zeros(len(tokens))
for i,token in enumerate(tokens):
  magnitudes[i] = np.sqrt( sum(embeddings[token,:]**2) )

plt.figure(figsize=(10,3))
plt.plot(magnitudes,'ks',markerfacecolor=[.6,.5,.2])
plt.gca().set(xlabel='Token index',ylabel='Embedding magnitude')
plt.show()

In [ ]:
# min-max scale the magnitudes
scaled_mags = (magnitudes - min(magnitudes)) / (max(magnitudes)-min(magnitudes))

plt.plot(magnitudes,scaled_mags,'ko',markerfacecolor=[.7,.9,.7],markersize=10)
plt.gca().set(xlabel='Raw magnitudes',ylabel='Scaled magnitudes')
plt.show()

In [ ]:
tokCount = 0

x_pos = 0  # starting x position (in axis coordinates)
y_pos = 1  # vertical center

fig, ax = plt.subplots(figsize=(10,2))
ax.axis('off')

for toki in range(len(tokens)):

  # text of this token
  toktext = tokenizer.decode([tokens[toki]])

  # width of the token
  token_width = en_width*len(toktext)

  # text object with background color matching the "activation"
  ax.text(x_pos+token_width/2, y_pos, toktext, fontsize=12, ha='center', va='center',fontfamily='monospace',
          bbox = dict(boxstyle='round,pad=.3', facecolor=mpl.cm.Reds(scaled_mags[toki]), edgecolor='none', alpha=.8))

  # update the token counter and x_pos
  tokCount += 1
  x_pos += token_width + .015 # plus a small gap

  # end of the line; reset coordinates and counter
  if tokCount>=20:
    y_pos -= .2
    x_pos = 0
    tokCount = 0

plt.show()

# Exercise 2: Cosine similarities in sequences

In [ ]:
# tokenize text
# https://en.wikipedia.org/wiki/Algae_fuel
text = "Algae fuel, algal biofuel, or algal oil is an alternative to liquid fossil fuels that use algae as the source of energy-rich oils. Also, algae fuels are an alternative to commonly known biofuel sources, such as corn and sugarcane. When made from seaweed (macroalgae) it can be known as seaweed fuel or seaweed oil.  These fuels have no practical significance but remain an aspirational target in the biofuels research area."
tokens = tokenizer.encode(text)[1:-1]

# get all similiarities
cossims = np.full(len(tokens),np.nan)

for i in range(1,len(tokens)):

  # previous token embedding
  v1 = embeddings[tokens[i-1],:].squeeze()
  v1norm = sum(v1**2)

  # current token embedding
  v2 = embeddings[tokens[i],:]
  v2norm = sum(v2**2)
  cossims[i] = abs( sum(v1*v2) / np.sqrt( v1norm*v2norm ) )

plt.figure(figsize=(10,3))
plt.plot(cossims,'ks',markerfacecolor=[.6,.5,.2])
plt.gca().set(xlabel='Token index',ylabel='Cosine similarity with previous token')
plt.show()

In [ ]:
# Do we need to scale after the loop?
# Couldn't we incorporate this into the previous for-loop?
scaled_cs = (cossims - np.nanmin(cossims)) / (np.nanmax(cossims)-np.nanmin(cossims))
print(scaled_cs)
scaled_cs[0] = 0

In [ ]:
tokCount = 0

x_pos = 0  # starting x position (in axis coordinates)
y_pos = 1  # vertical center

fig, ax = plt.subplots(figsize=(10,2))
ax.axis('off')

for toki in range(len(tokens)):

  # text of this token
  toktext = tokenizer.decode([tokens[toki]])

  # width of the token
  token_width = en_width*len(toktext)

  # text object with background color matching the "activation"
  ax.text(x_pos+token_width/2, y_pos, toktext, fontsize=12, ha='center', va='center',fontfamily='monospace',
          bbox = dict(boxstyle='round,pad=.3', facecolor=mpl.cm.Reds(scaled_cs[toki]), edgecolor='none', alpha=.8))

  # update the token counter and x_pos
  tokCount += 1
  x_pos += token_width + .015 # plus a small gap

  # end of the line; reset coordinates and counter
  if tokCount>=20:
    y_pos -= .2
    x_pos = 0
    tokCount = 0

plt.show()

# Exercise 3: Seeded cosine similarities

In [ ]:
# tokenize text
# https://en.wikipedia.org/wiki/Purple
text = "Purple has long been associated with royalty, originally because Tyrian purple dye—made from the secretions of sea snails—was extremely expensive in antiquity. Purple was the color worn by Roman magistrates; it became the imperial color worn by the rulers of the Byzantine Empire and the Holy Roman Empire, and later by Roman Catholic bishops. Similarly in Japan, the color is traditionally associated with the emperor and aristocracy."
tokens = tokenizer.encode(text)[1:-1]

# the "seed" token embedding
seedtok = tokenizer.encode('purple')[1:-1]
seedvect = embeddings[seedtok,:].squeeze()
seednorm = sum(seedvect**2)

# stored value is abs(cosine similarity) to seed
cossims = np.zeros(len(tokens))
for i,token in enumerate(tokens):
  targvect = embeddings[token,:]
  targnorm = sum(targvect**2)
  cossims[i] = abs( sum(seedvect*targvect) / np.sqrt( seednorm*targnorm ) )

plt.figure(figsize=(10,3))
plt.plot(cossims,'ks',markerfacecolor=[.9,.7,.9])
plt.gca().set(xlabel='Token index',ylabel='Sc with "purple"')
plt.show()

In [ ]:
# min-max scale
scaled_cs = (cossims - np.nanmin(cossims))/(np.nanmax(cossims)-np.nanmin(cossims))


tokCount = 0

x_pos = 0  # starting x position (in axis coordinates)
y_pos = 1  # vertical center

fig, ax = plt.subplots(figsize=(10,2))
ax.axis('off')

for toki in range(len(tokens)):

  # text of this token
  toktext = tokenizer.decode([tokens[toki]])

  # width of the token
  token_width = en_width*len(toktext)

  # text object with background color matching the "activation"
  ax.text(x_pos+token_width/2, y_pos, toktext, fontsize=12, ha='center', va='center',fontfamily='monospace',
          bbox = dict(boxstyle='round,pad=.3', facecolor=mpl.cm.Purples(cossims[toki]), edgecolor='none', alpha=.8))

  # update the token counter and x_pos
  tokCount += 1
  x_pos += token_width + .015 # plus a small gap

  # end of the line; reset coordinates and counter
  if tokCount>=20:
    y_pos -= .2
    x_pos = 0
    tokCount = 0

plt.show()